# Question answering on the SQuAD dataset

In [604]:
import sys
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
import gensim
import gensim.downloader as gloader
import transformers
import tokenizers
from transformers.trainer_pt_utils import nested_detach
from transformers.trainer_utils import EvalPrediction
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.normalizers import Sequence, StripAccents, Lowercase, Strip
from tokenizers.pre_tokenizers import Sequence as PreSequence
from tokenizers.pre_tokenizers import Whitespace, Punctuation

from IPython.display import display, HTML

import dataset
import model

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [452]:
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.xmargin'] = .05
plt.rcParams['axes.ymargin'] = .05
plt.style.use('ggplot')

%env WANDB_WATCH=all
%env WANDB_PROJECT=squad-qa
%env WANDB_DISABLED=false

env: WANDB_WATCH=all
env: WANDB_PROJECT=squad-qa
env: WANDB_DISABLED=false


In [643]:
!conda

/bin/sh: 1: conda: not found


In [54]:
!{sys.executable} -m wandb login

wandb: Currently logged in as: wadaboa (use `wandb login --relogin` to force relogin)


## Preliminaries

### Data loading/preparation

In [519]:
squad_dataset = dataset.SquadDataset()
squad_dataset.dataframe

,index,question,title,context_id,context,answer,answer_start,answer_end
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous,515,541
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",a copper statue of Christ,188,213
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",the Main Building,279,296
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",a Marian place of prayer and reflection,381,420
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",a golden statue of the Virgin Mary,92,126
...,...,...,...,...,...,...,...,...
87594,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,Kathmandu,18890,"Kathmandu Metropolitan City (KMC), in order to...",Oregon,229,235
87595,5735d259012e2f140011a09e,What was Yangon previously known as?,Kathmandu,18890,"Kathmandu Metropolitan City (KMC), in order to...",Rangoon,414,421
87596,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,Kathmandu,18890,"Kathmandu Metropolitan City (KMC), in order to...",Minsk,476,481
87597,5735d259012e2f140011a0a0,In what year did Kathmandu create its initial ...,Kathmandu,18890,"Kathmandu Metropolitan City (KMC), in order to...",1975,199,203


In [520]:
squad_dataset.train_df

,index,question,title,context_id,context,answer,answer_start,answer_end
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous,515,541
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",a copper statue of Christ,188,213
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",the Main Building,279,296
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",a Marian place of prayer and reflection,381,420
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",a golden statue of the Virgin Mary,92,126
...,...,...,...,...,...,...,...,...
70185,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,Kathmandu,18890,"Kathmandu Metropolitan City (KMC), in order to...",Oregon,229,235
70186,5735d259012e2f140011a09e,What was Yangon previously known as?,Kathmandu,18890,"Kathmandu Metropolitan City (KMC), in order to...",Rangoon,414,421
70187,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,Kathmandu,18890,"Kathmandu Metropolitan City (KMC), in order to...",Minsk,476,481
70188,5735d259012e2f140011a0a0,In what year did Kathmandu create its initial ...,Kathmandu,18890,"Kathmandu Metropolitan City (KMC), in order to...",1975,199,203


In [521]:
squad_dataset.val_df

,index,question,title,context_id,context,answer,answer_start,answer_end
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé,55,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269,286
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé,55,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207,226
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé,55,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526,530
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé,55,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166,180
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé,55,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276,286
...,...,...,...,...,...,...,...,...
17404,5732868bb3a91d1900202e0f,At what Augusta hole was the Eisenhower Pine l...,Dwight_D._Eisenhower,18592,"A loblolly pine, known as the ""Eisenhower Pine...",17th,74,78
17405,5732868bb3a91d1900202e10,How many meters away from the Masters tee on A...,Dwight_D._Eisenhower,18592,"A loblolly pine, known as the ""Eisenhower Pine...",192,110,113
17406,5732868bb3a91d1900202e11,What did Eisenhower want to be done to the Eis...,Dwight_D._Eisenhower,18592,"A loblolly pine, known as the ""Eisenhower Pine...",cut down,279,287
17407,5732868bb3a91d1900202e12,What damaged the Eisenhower Pine in February 2...,Dwight_D._Eisenhower,18592,"A loblolly pine, known as the ""Eisenhower Pine...",ice storm,478,487


In [522]:
squad_dataset.train_dataset[0]

('To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 515,
 541)

### GloVe embeddings

In [244]:
UNK_TOKEN = "[UNK]"
PAD_TOKEN = "[PAD]"
GLOVE_EMBEDDING_DIMENSION = 50

In [245]:
def load_embedding_model(model_type, embedding_dimension=50):
    """
    Loads a pre-trained word embedding model via gensim library
    """
    # Find the correct embedding model name
    download_path = ""
    if model_type.strip().lower() == "word2vec":
        download_path = "word2vec-google-news-300"
    elif model_type.strip().lower() == "glove":
        download_path = f"glove-wiki-gigaword-{embedding_dimension}"
    else:
        raise AttributeError(
            "Unsupported embedding model type (choose from {word2vec, glove})"
        )

    # Check download
    try:
        emb_model = gloader.load(download_path)
    except ValueError as e:
        print("Invalid embedding model name. Check the embedding dimension:")
        print("Word2Vec: {300}")
        print("GloVe: {50, 100, 200, 300}")
        raise e

    return emb_model


glove_embedding_model = load_embedding_model("glove", embedding_dimension=GLOVE_EMBEDDING_DIMENSION)

In [246]:
glove_unk = np.mean(glove_embedding_model.vectors, axis=0)
glove_embedding_model.add(UNK_TOKEN, glove_unk)

array([-0.12920076, -0.28866628, -0.01224866, -0.05676644, -0.20210965,
       -0.08389011,  0.33359843,  0.16045167,  0.03867431,  0.17833012,
        0.04696583, -0.00285802,  0.29099807,  0.04613704, -0.20923874,
       -0.06613114, -0.06822549,  0.07665912,  0.3134014 ,  0.17848536,
       -0.1225775 , -0.09916984, -0.07495987,  0.06413227,  0.14441176,
        0.60894334,  0.17463093,  0.05335403, -0.01273871,  0.03474107,
       -0.8123879 , -0.04688699,  0.20193407,  0.2031118 , -0.03935686,
        0.06967544, -0.01553638, -0.03405238, -0.06528071,  0.12250231,
        0.13991883, -0.17446303, -0.08011883,  0.0849521 , -0.01041659,
       -0.13705009,  0.20127155,  0.10069408,  0.00653003,  0.01685157],
      dtype=float32)

In [250]:
glove_embedding_model[UNK_TOKEN]

array([-0.12920076, -0.28866628, -0.01224866, -0.05676644, -0.20210965,
       -0.08389011,  0.33359843,  0.16045167,  0.03867431,  0.17833012,
        0.04696583, -0.00285802,  0.29099807,  0.04613704, -0.20923874,
       -0.06613114, -0.06822549,  0.07665912,  0.3134014 ,  0.17848536,
       -0.1225775 , -0.09916984, -0.07495987,  0.06413227,  0.14441176,
        0.60894334,  0.17463093,  0.05335403, -0.01273871,  0.03474107,
       -0.8123879 , -0.04688699,  0.20193407,  0.2031118 , -0.03935686,
        0.06967544, -0.01553638, -0.03405238, -0.06528071,  0.12250231,
        0.13991883, -0.17446303, -0.08011883,  0.0849521 , -0.01041659,
       -0.13705009,  0.20127155,  0.10069408,  0.00653003,  0.01685157],
      dtype=float32)

In [ ]:
list(glove_embedding_model.vocab.keys())[-1]

In [247]:
any(np.all(glove_embedding_model.vectors == 0, axis=1))

False

In [248]:
glove_embedding_model.add(PAD_TOKEN, np.zeros((1, GLOVE_EMBEDDING_DIMENSION)))
glove_embedding_model[PAD_TOKEN]

In [252]:
list(glove_embedding_model.vocab.keys())[-1]

'[PAD]'

In [249]:
glove_embedding_model.vectors.shape

(400002, 50)

In [253]:
glove_vocab = dict(
    zip(glove_embedding_model.index2word, range(len(glove_embedding_model.index2word)))
)

In [254]:
glove_embedding_layer = nn.Embedding(
    glove_embedding_model.vectors.shape[0],
    GLOVE_EMBEDDING_DIMENSION,
    padding_idx=glove_vocab[PAD_TOKEN],
)
glove_embedding_layer.weight = nn.Parameter(
    torch.from_numpy(glove_embedding_model.vectors)
)
glove_embedding_layer.weight.requires_grad = False

### Baseline tokenizer

In [529]:
MAX_CONTEXT_TOKENS = 300

In [530]:
baseline_question_tokenizer = Tokenizer(WordLevel(glove_vocab, unk_token=UNK_TOKEN))
baseline_question_tokenizer.normalizer = Sequence([StripAccents(), Lowercase(), Strip()])
baseline_question_tokenizer.pre_tokenizer = PreSequence([Whitespace(), Punctuation()])
baseline_question_tokenizer.enable_padding(
    direction="right",
    pad_id=glove_vocab[PAD_TOKEN],
    pad_type_id=1,
    pad_token=PAD_TOKEN
)

baseline_context_tokenizer = Tokenizer(WordLevel(glove_vocab, unk_token=UNK_TOKEN))
baseline_context_tokenizer.normalizer = Sequence([StripAccents(), Lowercase(), Strip()])
baseline_context_tokenizer.pre_tokenizer = PreSequence([Whitespace(), Punctuation()])
baseline_context_tokenizer.enable_padding(
    direction="right",
    pad_id=glove_vocab[PAD_TOKEN],
    pad_type_id=1,
    pad_token=PAD_TOKEN,
    length=MAX_CONTEXT_TOKENS,
)
baseline_context_tokenizer.enable_truncation(MAX_CONTEXT_TOKENS)

### Pre-processing

In [501]:
def lost_answers_indexes(df, tokenized_contexts):
    whole_answers_start = df["answer_start"].tolist()
    whole_answers_end = df["answer_end"].tolist()
    lost_dirty, lost_truncated = [], []
    for i, (c, s, e) in enumerate(
        zip(tokenized_contexts, whole_answers_start, whole_answers_end)
    ):
        offsets = torch.tensor(c.offsets)[torch.tensor(c.attention_mask).bool()]
        start_index = torch.nonzero(offsets[:, 0] == s)
        end_index = torch.nonzero(offsets[:, 1] == e)
        if len(start_index) == 0 or len(end_index) == 0:
            if s > offsets[-1, 0] or e > offsets[-1, 1]:
                lost_truncated.append(i)
            else:
                lost_dirty.append(i)

    return lost_truncated, lost_dirty


tokenized_contexts = baseline_context_tokenizer.encode_batch(
    squad_dataset.dataframe["context"].tolist()
)
lost_truncated, lost_dirty = lost_answers_indexes(
    squad_dataset.dataframe, tokenized_contexts
)

In [502]:
lost_truncated_perc = len(lost_truncated) / len(squad_dataset.dataframe)
lost_dirty_perc = len(lost_dirty) / len(squad_dataset.dataframe)
print(
    f"Lost {len(lost_truncated)}/{len(squad_dataset.dataframe)} ({lost_truncated_perc:.2%}) answers, because of truncation"
)
print(
    f"Lost {len(lost_dirty)}/{len(squad_dataset.dataframe)} ({lost_dirty_perc:.2%}) answers, because of dirtyness"
)

Lost 183/87599 (0.21%) answers, because of truncation
Lost 260/87599 (0.30%) answers, because of dirtyness


In [503]:
def show_df_row(df, index):
    row = df.iloc[index]
    display(HTML(pd.DataFrame([row]).to_html()))

In [510]:
random_trunc_index = random.choice(lost_truncated)
show_df_row(squad_dataset.dataframe, random_trunc_index)

,index,question,title,context_id,context,answer,answer_start,answer_end
73891,572fdc43947a6a140053cd76,What country is spoken of in the inscriptions of a country in Northern Africa from 1390-1352 BC ?,Greeks,16034,"In Homer's Iliad, the names Danaans (or Danaoi: Δαναοί) and Argives (Argives: Αργείοι) are used to designate the Greek forces opposed to the Trojans. The myth of Danaus, whose origin is Egypt, is a foundation legend of Argos. His daughters Danaides, were forced in Tartarus to carry a jug to fill a bathtub without a bottom. This myth is connected with a task that can never be fulfilled (Sisyphos) and the name can be derived from the PIE root *danu: ""river"". There is not any satisfactory theory on their origin. Some scholars connect Danaans with the Denyen, one of the groups of the sea peoples who attacked Egypt during the reign of Ramesses III (1187-1156 BCE). The same inscription mentions the Weshesh who might have been the Achaeans. The Denyen seem to have been inhabitants of the city Adana in Cilicia. Pottery similar to that of Mycenae itself has been found in Tarsus of Cilicia and it seems that some refugees from the Aegean went there after the collapse of the Mycenean civilization. These Cilicians seem to have been called Dananiyim, the same word as Danaoi who attacked Egypt in 1191 BC along with the Quaouash (or Weshesh) who may be Achaeans. They were also called Danuna according to a Hittite inscription and the same name is mentioned in the Amarna letters. Julius Pokorny reconstructs the name from the PIE root da:-: ""flow, river"", da:-nu: ""any moving liquid, drops"", da: navo ""people living by the river, Skyth. nomadic people (in Rigveda water-demons, fem.Da:nu primordial goddess), in Greek Danaoi, Egypt. Danuna"". It is also possible that the name Danaans is pre-Greek. A country Danaja with a city Mukana (propaply: Mycenea) is mentioned in inscriptions from Egypt from Amenophis III (1390-1352 BC), Thutmosis III (1437 BC).",A country Danaja with a city Mukana (propaply: Mycenea) is mentioned in inscriptions from Egypt,1601,1696


In [513]:
random_dirty_index = random.choice(lost_dirty)
show_df_row(squad_dataset.dataframe, random_dirty_index)

,index,question,title,context_id,context,answer,answer_start,answer_end
58329,5727e7654b864d1900163faf,What were the nuns allowed to minimally do?,Dominican_Order,12565,"Women could not be professed to the Dominican religious life before the age of thirteen. The formula for profession contained in the Constitutions of Montargis Priory (1250) demands that nuns pledge obedience to God, the Blessed Virgin, their prioress and her successors according to the Rule of St. Augustine and the institute of the order, until death. The clothing of the sisters consisted of a white tunic and scapular, a leather belt, a black mantle, and a black veil. Candidates to profession were tested to reveal whether they were actually married women who had merely separated from their husbands. Their intellectual abilities were also tested. Nuns were to be silent in places of prayer, the cloister, the dormitory, and refectory. Silence was maintained unless the prioress granted an exception for a specific cause. Speaking was allowed in the common parlor, but it was subordinate to strict rules, and the prioress, subprioress or other senior nun had to be present.",Speak,829,834


In [527]:
to_remove = lost_truncated + lost_dirty
df = squad_dataset.dataframe.drop(to_remove)
assert len(df) == len(squad_dataset.dataframe) - len(to_remove), (
    f"Before {len(squad_dataset.dataframe)},
    f"After {len(df)}",
    f"Removed {len(to_remove)}",
)

In [528]:
squad_dataset.update_df(df)
squad_dataset.dataframe

,index,question,title,context_id,context,answer,answer_start,answer_end
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous,515,541
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",a copper statue of Christ,188,213
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",the Main Building,279,296
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",a Marian place of prayer and reflection,381,420
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",a golden statue of the Virgin Mary,92,126
...,...,...,...,...,...,...,...,...
87594,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,Kathmandu,18890,"Kathmandu Metropolitan City (KMC), in order to...",Oregon,229,235
87595,5735d259012e2f140011a09e,What was Yangon previously known as?,Kathmandu,18890,"Kathmandu Metropolitan City (KMC), in order to...",Rangoon,414,421
87596,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,Kathmandu,18890,"Kathmandu Metropolitan City (KMC), in order to...",Minsk,476,481
87597,5735d259012e2f140011a0a0,In what year did Kathmandu create its initial ...,Kathmandu,18890,"Kathmandu Metropolitan City (KMC), in order to...",1975,199,203


### Metrics and logs

In [630]:
def exact_match(labels, preds):
    assert labels.shape == preds.shape
    total = labels.shape[0]
    matches = np.count_nonzero((labels == preds).all(axis=1))
    return matches / total

In [631]:
def compute_metrics(eval_prediction):
    # Labels are stored as a tuple of tensors
    # (one for answer start and one for answer end)
    labels = torch.cat(
        [
            eval_prediction.label_ids[0].view(-1, 1),
            eval_prediction.label_ids[1].view(-1, 1),
        ],
        dim=1,
    ).numpy()
    preds = eval_prediction.predictions.numpy()
    f_labels, f_preds = labels.flatten(), preds.flatten()
    return {
        "f1": sklearn.metrics.f1_score(f_labels, f_preds, average="macro"),
        "accuracy": sklearn.metrics.accuracy_score(f_labels, f_preds),
        "em": exact_match(labels, preds),
    }

In [632]:
class SquadTrainer(transformers.Trainer):
    def __init__(self, *args, **kwargs):
        super(SquadTrainer, self).__init__(*args, **kwargs)

    def compute_loss(self, model, inputs):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        outputs = model(**inputs)

        has_labels = all(inputs.get(k) is not None for k in self.label_names)
        if has_labels:
            labels = nested_detach(tuple(inputs.get(name) for name in self.label_names))
            if len(labels) == 1:
                labels = labels[0]
        else:
            labels = None

        if labels is not None:
            metrics = self.compute_metrics(
                EvalPrediction(predictions=outputs["outputs"], label_ids=labels)
            )
            self.log(metrics)

        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]
        # We don't use .loss here since the model may return tuples instead of ModelOutput.
        return outputs["loss"] if isinstance(outputs, dict) else outputs[0]

## Baseline

In [633]:
class BaselineDataCollatorWithPadding:
    """
    Data collator that will dynamically pad the inputs received
    """

    def __init__(self, question_tokenizer, context_tokenizer):
        self.question_tokenizer = question_tokenizer
        self.context_tokenizer = context_tokenizer

    def find_tokenized_answer_indexes(self, offsets, query, dim):
        index = torch.nonzero(offsets[:, dim] == query)
        assert len(index) in (0, 1)
        return index.item() if len(index) > 0 else -1

    def __call__(self, inputs):
        (questions, contexts, answers_start, answers_end) = zip(*inputs)
        tokenized_questions = self.question_tokenizer.encode_batch(questions)
        tokenized_contexts = self.context_tokenizer.encode_batch(contexts)
        batch_size = len(tokenized_questions)
        questions_shape = (batch_size, len(tokenized_questions[0].ids))
        contexts_shape = (batch_size, len(tokenized_contexts[0].ids))

        batch = {
            "question_ids": torch.empty(questions_shape, dtype=torch.long),
            "question_type_ids": torch.empty(questions_shape, dtype=torch.long),
            "question_attention_mask": torch.empty(questions_shape, dtype=torch.bool),
            "question_special_tokens_mask": torch.empty(
                questions_shape, dtype=torch.bool
            ),
            "question_lenghts": torch.empty((batch_size,), dtype=torch.long),
            "context_ids": torch.empty(contexts_shape, dtype=torch.long),
            "context_type_ids": torch.empty(contexts_shape, dtype=torch.long),
            "context_attention_mask": torch.empty(contexts_shape, dtype=torch.bool),
            "context_special_tokens_mask": torch.empty(
                contexts_shape, dtype=torch.bool
            ),
            "context_offsets": torch.empty((*contexts_shape, 2), dtype=torch.long),
            "context_lenghts": torch.empty((batch_size,), dtype=torch.long),
            # "answer_start": torch.tensor(answers_start, dtype=torch.long),
            # "answer_end": torch.tensor(answers_end, dtype=torch.long),
            "answer_start": torch.empty((batch_size,), dtype=torch.long),
            "answer_end": torch.empty((batch_size,), dtype=torch.long),
        }
        for i in range(batch_size):
            batch["question_ids"][i] = torch.tensor(tokenized_questions[i].ids)
            batch["question_type_ids"][i] = torch.tensor(
                tokenized_questions[i].type_ids
            )
            batch["question_attention_mask"][i] = torch.tensor(
                tokenized_questions[i].attention_mask
            )
            batch["question_special_tokens_mask"][i] = torch.tensor(
                tokenized_questions[i].special_tokens_mask
            )
            batch["question_lenghts"][i] = torch.count_nonzero(
                ~batch["question_special_tokens_mask"][i]
            )
            batch["context_ids"][i] = torch.tensor(tokenized_contexts[i].ids)
            batch["context_type_ids"][i] = torch.tensor(tokenized_contexts[i].type_ids)
            batch["context_attention_mask"][i] = torch.tensor(
                tokenized_contexts[i].attention_mask
            )
            batch["context_special_tokens_mask"][i] = torch.tensor(
                tokenized_contexts[i].special_tokens_mask
            )
            batch["context_offsets"][i] = torch.tensor(tokenized_contexts[i].offsets)
            batch["context_lenghts"][i] = torch.count_nonzero(
                ~batch["context_special_tokens_mask"][i]
            )
            masked_offsets = batch["context_offsets"][i][batch["context_attention_mask"][i]]
            batch["answer_start"][i] = self.find_tokenized_answer_indexes(
                masked_offsets, answers_start[i], 0
            )
            batch["answer_end"][i] = self.find_tokenized_answer_indexes(
                masked_offsets, answers_end[i], 1
            )

        return batch

### Baseline model

In [634]:
baseline_model = model.QABaselineModel(glove_embedding_layer, MAX_CONTEXT_TOKENS)
baseline_model.count_parameters()

101400

In [635]:
baseline_args = transformers.TrainingArguments(
    output_dir="./checkpoints",
    logging_dir="./runs",
    logging_first_step=True,
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=1e-3,
    num_train_epochs=10,
    remove_unused_columns=False,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=64,
    label_names=["answer_start", "answer_end"],
)

In [636]:
baseline_trainer = SquadTrainer(
    model=baseline_model,
    args=baseline_args,
    data_collator=BaselineDataCollatorWithPadding(
        baseline_question_tokenizer, baseline_context_tokenizer
    ),
    train_dataset=squad_dataset.train_dataset,
    eval_dataset=squad_dataset.val_dataset,
    compute_metrics=compute_metrics,
    # callbacks=[transformers.integrations.WandbCallback]
)

In [637]:
baseline_trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

### BiDAF

In [354]:
bidaf_model = model.BiDAFModel(glove_embedding_layer)
bidaf_model.count_parameters()

314350

In [355]:
bidaf_args = transformers.TrainingArguments(
    output_dir="./checkpoints",
    logging_dir="./runs",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    num_train_epochs=3,
    remove_unused_columns=False,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=64,
    label_names=["answer_start", "answer_end"]
)

In [356]:
bidaf_optimizer = optim.Adadelta(bidaf_model.parameters(), lr=0.5)
bidaf_lr_scheduler = optim.lr_scheduler.ExponentialLR(bidaf_optimizer, gamma=.999)

In [357]:
bidaf_trainer = transformers.Trainer(
    model=bidaf_model,
    args=bidaf_args,
    data_collator=BaselineDataCollatorWithPadding(baseline_question_tokenizer, baseline_context_tokenizer),
    train_dataset=squad_dataset.train_dataset,
    eval_dataset=squad_dataset.val_dataset,
    optimizers=(bidaf_optimizer, bidaf_lr_scheduler),
    #callbacks=[transformers.integrations.WandbCallback]
)

In [358]:
baseline_trainer.train()

torch.Size([256, 300, 1]) torch.Size([256, 300, 1])
tensor(15.0205, grad_fn=<NllLossBackward>) tensor(13.8060, grad_fn=<NllLossBackward>)


KeyboardInterrupt: 

## BERT

In [277]:
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased")
model = transformers.BertModel.from_pretrained("bert-base-uncased")

In [58]:
args = transformers.TrainingArguments(
    output_dir="./checkpoints",
    logging_dir="./runs",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    remove_unused_columns=False,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    label_names=["answer_start", "answer_end"],
)

In [59]:
class DataCollatorWithPadding:
    """
    Data collator that will dynamically pad the inputs received
    """

    def __init__(self, tokenizer, padding=True, max_lenght=None):
        self.tokenizer = tokenizer
        self.padding = True
        self.max_length = None

    def __call__(self, inputs):
        (questions, contexts, _, _) = zip(*inputs)
        tokenized = self.tokenizer(questions, contexts)
        batch = self.tokenizer.pad(
            tokenized,
            padding=self.padding,
            max_length=self.max_length,
            return_tensors="pt",
        )
        return batch

In [60]:
data_collator = DataCollatorWithPadding(tokenizer)

NameError: name 'tokenizer' is not defined

In [ ]:
trainer = transformers.Trainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=squad_dataset.train_dataset,
    eval_dataset=squad_dataset.val_dataset
)

In [288]:
trainer.train()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


KeyError: 'loss'